In [ ]:
# INSTALL OR-TOOLS THAT CAN RUN ON WINDOWS + JUPYTER + ANACONDA
import sys
!{sys.executable} -m pip install --upgrade --no-cache-dir ortools

In [ ]:
# 20 Jobs_CP model
from ortools.sat.python import cp_model
import pandas as pd

def load_MFJS20_from_excel(path="20 JB.xlsx", sheet="CP"):
    nbJobs = 20
    nbProcess = 8
    nbMchs = 28
    
    # Machine production power (kW)
    machine_run_power = [
        1.5,1.5,1.5,        # M1-3 
        3.0,3.0,3.0,        # M4-6 
        2.0,2.0,2.0,        # M7-9 
        1.0,1.0,1.0,        # M10-12 
        1.0,1.0,1.0,        # M13-15
        1.5,1.5,            # M16-17 
        2.5,                # M18    
        1.0,1.0,1.0,1.0,1.0,# M19-23 
        2.0,2.0,2.0,2.0,2.0 # M24-28 
    ]

    # Idle power (kW)
    D_k = [
        0.75, 0.75, 0.75,   # M1-3
        1.5, 1.5, 1.5,      # M4-6
        1.0, 1.0, 1.0,      # M7-9
        0.5, 0.5, 0.5,      # M10-12
        0.5, 0.5, 0.5,      # M13-15
        0.75, 0.75,         # M16-17
        1.25,               # M18
        0.5, 0.5, 0.5, 0.5, 0.5,  # M19-23
        1.0, 1.0, 1.0, 1.0, 1.0   # M24-28
    ]

    # Shutdown energy (kWh)
    G_k = [
        2.0, 2.0, 2.0,      # M1-3
        4.0, 4.0, 4.0,      # M4-6
        2.7, 2.7, 2.7,      # M7-9
        1.3, 1.3, 1.3,      # M10-12
        1.3, 1.3, 1.3,      # M13-15
        2.0, 2.0,           # M16-17
        4.0,                # M18
        1.3, 1.3, 1.3, 1.3, 1.3,  # M19-23
        2.7, 2.7, 2.7, 2.7, 2.7   # M24-28
    ]
    # Breakeven time (hour)
    TB_hours = (
        [2.7]*9 +    # M1-9
        [2.6]*6 +    # M10-15
        [2.7]*2 +    # M16-17
        [3.2]  +     # M18
        [2.6]*5 +    # M19-23
        [2.7]*5      # M24-28
    )
    TB = [int(t*60) for t in TB_hours] 		 # minute
    
   # Load data from Excel
    raw = pd.read_excel(path, sheet_name=sheet)
    header = raw.iloc[0].tolist()
    df = raw.iloc[1:].copy()
    df.columns = header
    
    job_col = "Job " if "Job " in df.columns else "Job"
    op_col = "Operation"
    df[job_col] = df[job_col].astype(float).ffill().astype(int)
    df[op_col] = df[op_col].astype(int)
    
    machine_cols = [f"M{i}" for i in range(1, nbMchs+1)]
    x_raw = [[[0]*nbProcess for _ in range(nbJobs)] for _ in range(nbMchs)]
    ptime_raw = [[[0]*nbProcess for _ in range(nbJobs)] for _ in range(nbMchs)]
    power_raw = [[[0]*nbProcess for _ in range(nbJobs)] for _ in range(nbMchs)]
    
    for _, row in df.iterrows():
        j = row[job_col] - 1
        o = row[op_col] - 1
        if o < 0 or o >= nbProcess:
            continue
        for m_idx, mcol in enumerate(machine_cols):
            val = row[mcol]
            if pd.notna(val) and float(val) > 0:
                x_raw[m_idx][j][o] = 1
                ptime_raw[m_idx][j][o] = int(val)
                power_raw[m_idx][j][o] = machine_run_power[m_idx]

    # Extract processing times and machine capabilities from Excel
    import copy
    x = copy.deepcopy(x_raw)
    ptime = copy.deepcopy(ptime_raw)
    power = copy.deepcopy(power_raw)

    # Calculate actual number of operations per job
    actual_ops_per_job = [0] * nbJobs
    for j in range(nbJobs):
        for o in range(nbProcess):
            if any(x[m][j][o] == 1 for m in range(nbMchs)):
                actual_ops_per_job[j] = max(actual_ops_per_job[j], o + 1)
  
    # Identify dummy operations              
    dummy_ops = set()
    for j in range(nbJobs):
        for o in range(actual_ops_per_job[j], nbProcess):
            dummy_ops.add((j, o))
    
    SCALE = 100
    power_scaled = [[[int(p*SCALE) for p in row] for row in mach] for mach in power]
    D_k = [int(d*SCALE) for d in D_k]
    G_k = [int(g*SCALE) for g in G_k]
    P0 = int(10*SCALE)
    
    return {
        "nbJobs": nbJobs,
        "nbProcess": nbProcess,
        "nbMchs": nbMchs,
        "x": x,
        "ptime": ptime,
        "power_scaled": power_scaled,
        "actual_ops_per_job": actual_ops_per_job,
        "dummy_ops": dummy_ops,
        "TB": TB,
        "D_k": D_k,
        "G_k": G_k,
        "P0": P0,
        "SCALE": SCALE,
        "N_k": [5]*nbMchs
    }

def build_cp2_fixed(data):
    nbJobs = data['nbJobs']
    nbProcess = data['nbProcess']
    nbMchs = data['nbMchs']
    TB = data['TB']            
    D_k = data['D_k']           
    G_k = data['G_k']           
    P0 = data['P0']             
    SCALE = data['SCALE']
    N_k = data['N_k']
    x = data['x']
    ptime = data['ptime']
    power_scaled = data['power_scaled']
    dummy_ops = data['dummy_ops']
    actual_ops_per_job = data['actual_ops_per_job']
    
    if len(x) < nbMchs or len(ptime) < nbMchs:
        print(f"DATA ERROR: Lack of machine data!")
        return None
    
    model = cp_model.CpModel()
    
   # Build capability dict (j,o,m)
    can_process = {}
    proc_time = {}
    power_int = {} 
    
    for m in range(nbMchs):
        for j in range(nbJobs):
            for o in range(nbProcess):
                if x[m][j][o] == 1 and ptime[m][j][o] > 0 and (j,o) not in dummy_ops:
                    key = (j+1, o+1, m+1)
                    can_process[key] = True
                    proc_time[key] = int(ptime[m][j][o]) 
                    power_int[key] = int(power_scaled[m][j][o])
    
    raw_horizon = 0
    for j in range(nbJobs):
        job_sum = sum(max(ptime[m][j][o] for m in range(nbMchs)) for o in range(nbProcess))
        raw_horizon = max(raw_horizon, job_sum)
    HORIZON = raw_horizon + 1000
    MAX_ENERGY_VAL = 10**10
    
    # Create decision variables
    mode_iv = {}
    mode_pres = {}
    mode_start = {}
    mode_end = {}
    
    for (j,o,m) in can_process:
        dur = proc_time[(j,o,m)]
        s = model.NewIntVar(0, HORIZON, f"s_{j}_{o}_{m}")
        e = model.NewIntVar(0, HORIZON, f"e_{j}_{o}_{m}")
        p = model.NewBoolVar(f"p_{j}_{o}_{m}")
        iv = model.NewOptionalIntervalVar(s, dur, e, p, f"iv_{j}_{o}_{m}")
        
        mode_iv[(j,o,m)] = iv
        mode_pres[(j,o,m)] = p
        mode_start[(j,o,m)] = s
        mode_end[(j,o,m)] = e
    
    op_start, op_end = {}, {}
# Each operation assigned to exactly one machine
    for j in range(1, nbJobs+1):
        for o in range(1, nbProcess+1):
            cands = [(j,o,m) for m in range(1, nbMchs+1) if (j,o,m) in mode_iv]
            if not cands: continue
            model.Add(sum(mode_pres[k] for k in cands) == 1)
            s = model.NewIntVar(0, HORIZON, f"op_s_{j}_{o}")
            e = model.NewIntVar(0, HORIZON, f"op_e_{j}_{o}")
            
            for cand in cands:
                model.Add(s == mode_start[cand]).OnlyEnforceIf(mode_pres[cand])
                model.Add(e == mode_end[cand]).OnlyEnforceIf(mode_pres[cand])
            op_start[(j,o)] = s
            op_end[(j,o)] = e
    # Precedence
    for j in range(1, nbJobs+1):
        for o in range(1, actual_ops_per_job[j-1]):
            if (j,o) in op_end and (j,o+1) in op_start:
                model.Add(op_end[(j,o)] <= op_start[(j,o+1)])
    # no-overlap constraints
    machine_ops = {m: [] for m in range(1, nbMchs+1)}
    for (j,o,m) in mode_iv:
        machine_ops[m].append((j,o,m))
    energy_idle_terms = []
    energy_sd_terms = []
    
    for m in range(1, nbMchs+1):
        ops = machine_ops[m]
        if ops:
            model.AddNoOverlap([mode_iv[k] for k in ops])	# No overlap
        if len(ops) < 2: 
            continue
        
        # Track gaps between operations
        next_op = {}
       
        for i, op_a in enumerate(ops):
            for j_idx, op_b in enumerate(ops):
                if i != j_idx:
                    is_next = model.NewBoolVar(f"next_{m}_{i}_{j_idx}")
                    next_op[(op_a, op_b)] = is_next
      
        # Ensure both operations are selected if one follows the other
        for (op_a, op_b), is_next in next_op.items():
            model.AddImplication(is_next, mode_pres[op_a])
            model.AddImplication(is_next, mode_pres[op_b])
       
        # Each operation has at most 1 predecessor and 1 successor
        for op_a in ops:
            out_nexts = [next_op[(op_a, op_b)] for op_b in ops if op_a != op_b]
            if out_nexts:
                model.Add(sum(out_nexts) <= 1)
           
            in_prevs = [next_op[(op_b, op_a)] for op_b in ops if op_a != op_b]
            if in_prevs:
                model.Add(sum(in_prevs) <= 1)
       
        # Identify first and last operations in sequence
        is_firsts = []
        is_lasts = []
       
        for i, op_a in enumerate(ops):
            prevs = [next_op[(op_b, op_a)] for j_idx, op_b in enumerate(ops) if i != j_idx]
            is_first = model.NewBoolVar(f"is_first_{m}_{i}")
            is_firsts.append(is_first)
            model.Add(sum(prevs) + is_first == mode_pres[op_a])
           
            nexts = [next_op[(op_a, op_b)] for j_idx, op_b in enumerate(ops) if i != j_idx]
            is_last = model.NewBoolVar(f"is_last_{m}_{i}")
            is_lasts.append(is_last)
            # Use == instead of >=
            model.Add(sum(nexts) + is_last == mode_pres[op_a])
       
        # Enforce exactly one first và one last if machine is used
        num_selected = model.NewIntVar(0, len(ops), f"num_sel_M{m}")
        model.Add(num_selected == sum(mode_pres[op_a] for op_a in ops))
       
        machine_used = model.NewBoolVar(f"used_M{m}")
        model.Add(num_selected > 0).OnlyEnforceIf(machine_used)
        model.Add(num_selected == 0).OnlyEnforceIf(machine_used.Not())
       
        model.Add(sum(is_firsts) == 1).OnlyEnforceIf(machine_used)
        model.Add(sum(is_firsts) == 0).OnlyEnforceIf(machine_used.Not())
       
        model.Add(sum(is_lasts) == 1).OnlyEnforceIf(machine_used)
        model.Add(sum(is_lasts) == 0).OnlyEnforceIf(machine_used.Not())
        
        # Calculate idle and shutdown energy
        shutdown_count_terms = []
        
        for (op_a, op_b), is_next in next_op.items():
            model.Add(mode_end[op_a] <= mode_start[op_b]).OnlyEnforceIf(is_next)
           
            # Calculate gap
            gap = model.NewIntVar(0, HORIZON, f"gap_{m}_{op_a}_{op_b}")
            model.Add(gap == mode_start[op_b] - mode_end[op_a]).OnlyEnforceIf(is_next)
            model.Add(gap == 0).OnlyEnforceIf(is_next.Not())
            
            # Compare gap with TB_k
            tb_val = int(TB[m-1])
            gap_large = model.NewBoolVar(f"gap_large_{m}_{op_a}_{op_b}")
            model.Add(gap >= tb_val).OnlyEnforceIf(gap_large)
            model.Add(gap < tb_val).OnlyEnforceIf(gap_large.Not())
            
            # Shutdown decision
            is_shutdown = model.NewBoolVar(f"shutdown_{m}_{op_a}_{op_b}")
            model.AddBoolAnd([is_next, gap_large]).OnlyEnforceIf(is_shutdown)
            model.AddBoolOr([is_next.Not(), gap_large.Not()]).OnlyEnforceIf(is_shutdown.Not())
            shutdown_count_terms.append(is_shutdown)
            
            # Idle decision
            is_idle = model.NewBoolVar(f"idle_{m}_{op_a}_{op_b}")
            model.AddBoolAnd([is_next, is_shutdown.Not()]).OnlyEnforceIf(is_idle)
            model.AddBoolOr([is_next.Not(), is_shutdown]).OnlyEnforceIf(is_idle.Not())
            
            # Calculate idle energy
            dk_val = int(D_k[m-1])
            temp_idle = model.NewIntVar(0, min(HORIZON * dk_val, 10**10), f"temp_idle_{m}_{op_a}_{op_b}")
            model.Add(temp_idle == dk_val * gap).OnlyEnforceIf(is_idle)
            model.Add(temp_idle == 0).OnlyEnforceIf(is_idle.Not())
            
            idle_energy = model.NewIntVar(0, MAX_ENERGY_VAL, f"idle_E_{m}_{op_a}_{op_b}")
            model.AddDivisionEquality(idle_energy, temp_idle, 60)
            energy_idle_terms.append(idle_energy)
        
        # Calculate shutdown energy for machine m
        if shutdown_count_terms:
            shut_count = model.NewIntVar(0, int(N_k[m-1]), f"sdCnt_M{m}")
            model.Add(shut_count == sum(shutdown_count_terms))
            model.Add(shut_count <= int(N_k[m-1]))
            
            shut_energy = model.NewIntVar(0, MAX_ENERGY_VAL, f"sdE_M{m}")
            model.Add(shut_energy == shut_count * int(G_k[m-1]))
            energy_sd_terms.append(shut_energy)
    
# OBJECTIVE FUNCTION: Minimize total energy consumption

    # Production energy
    energy_prod = model.NewIntVar(0, MAX_ENERGY_VAL, 'E_prod')
    prod_terms = []
    for k in mode_pres:
        temp = model.NewIntVar(0, MAX_ENERGY_VAL, f"temp_prod_{k}")
        model.Add(temp == power_int[k] * proc_time[k]).OnlyEnforceIf(mode_pres[k])
        model.Add(temp == 0).OnlyEnforceIf(mode_pres[k].Not())
        pe = model.NewIntVar(0, MAX_ENERGY_VAL, f"pe_{k}")
        model.AddDivisionEquality(pe, temp, 60)
        prod_terms.append(pe)
    model.Add(energy_prod == sum(prod_terms))

    # Idle energy
    energy_idle = model.NewIntVar(0, MAX_ENERGY_VAL, 'E_idle')
    if energy_idle_terms: 
        model.Add(energy_idle == sum(energy_idle_terms))
    else: 
        model.Add(energy_idle == 0)
        
    # Shutdown energy
    energy_sd = model.NewIntVar(0, MAX_ENERGY_VAL, 'E_sd')
    if energy_sd_terms: 
        model.Add(energy_sd == sum(energy_sd_terms))
    else: 
        model.Add(energy_sd == 0)
        
    # Makespan
    makespan = model.NewIntVar(0, HORIZON, 'makespan')
    if len(mode_end) > 0:
        model.AddMaxEquality(makespan, list(mode_end.values()))
    else:
        model.Add(makespan == 0)
    # Common energy
    energy_common = model.NewIntVar(0, MAX_ENERGY_VAL, 'E_common')
    temp_common = model.NewIntVar(0, MAX_ENERGY_VAL, 'temp_common')
    model.Add(temp_common == makespan * int(P0))
    model.AddDivisionEquality(energy_common, temp_common, 60)
    
    # Total energy
    energy_total = model.NewIntVar(0, MAX_ENERGY_VAL * 4, 'E_total')
    model.Add(energy_total == energy_prod + energy_idle + energy_sd + energy_common)
    
    model.Minimize(energy_total)
    
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 1800
    
    return (model, solver, energy_prod, energy_idle, energy_sd, energy_common, energy_total, makespan, mode_pres, mode_start, mode_end, proc_time, power_int)
    
if __name__ == "__main__":
    data = load_MFJS20_from_excel("20 JB.xlsx")
    result = build_cp2_fixed(data)
    
    if result is None:
        print("Failed to build model")
        exit(1)
    
    model, solver, E_prod, E_idle, E_sd, E_common, E_total, makespan, mode_pres, mode_start, mode_end, proc_time, power_int = result
    
    print("Solving...")
    status = solver.Solve(model)
    
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        scale = data['SCALE']
        
        print('\n' + "="*80)
        print(' ENERGY RESULTS')
        print("="*80)
        print(f"{'Metric':<30} {'Scaled Value':<20} {'Real Value':<20}")
        print("-"*80)
        print(f"{'Makespan':<30} {solver.Value(makespan):<20} {solver.Value(makespan):.2f} minutes")
        print(f"{'Production Energy':<30} {solver.Value(E_prod):<20} {solver.Value(E_prod)/scale:.2f} kWh")
        print(f"{'Idle Energy':<30} {solver.Value(E_idle):<20} {solver.Value(E_idle)/scale:.2f} kWh")
        print(f"{'Shutdown Energy':<30} {solver.Value(E_sd):<20} {solver.Value(E_sd)/scale:.2f} kWh")
        print(f"{'Common Energy':<30} {solver.Value(E_common):<20} {solver.Value(E_common)/scale:.2f} kWh")
        print("-"*80)
        print(f"{'TOTAL ENERGY':<30} {solver.Value(E_total):<20} {solver.Value(E_total)/scale:.2f} kWh")
        print("="*80)
        
        schedule = []
        for (j, o, m) in mode_pres:
            if solver.Value(mode_pres[(j, o, m)]):
                start = solver.Value(mode_start[(j, o, m)])
                end = solver.Value(mode_end[(j, o, m)])
                duration = proc_time[(j, o, m)]
                power = power_int[(j, o, m)] / scale
                schedule.append({
                    'job': j,
                    'operation': o,
                    'machine': m,
                    'start': start,
                    'end': end,
                    'duration': duration,
                    'power': power
                })
        
        schedule.sort(key=lambda x: (x['job'], x['operation']))
        
        print('\n' + "="*80)
        print(' DETAILED SCHEDULE')
        print("="*80)
        
        for j in range(1, data['nbJobs'] + 1):
            job_ops = [s for s in schedule if s['job'] == j]
            if job_ops:
                print(f"\n{'─'*80}")
                print(f"JOB {j}")
                print(f"{'─'*80}")
                print(f"{'Op':<5} {'Machine':<10} {'Start':<12} {'End':<12} {'Duration':<12} {'Power':<10}")
                print(f"{'-'*80}")
                
                for op in job_ops:
                    print(f"{op['operation']:<5} M{op['machine']:<9} {op['start']:<12.0f} {op['end']:<12.0f} {op['duration']:<12.0f} {op['power']:<10.2f} kW")

        # Machine utilization
        print('\n' + "="*80)
        print(' MACHINE UTILIZATION')
        print("="*80)
       
        schedule_by_machine = sorted(schedule, key=lambda x: (x['machine'], x['start']))
       
        current_machine = None
        for op in schedule_by_machine:
            if op['machine'] != current_machine:
                if current_machine is not None:
                    print()
                current_machine = op['machine']
                print(f"\nMachine M{current_machine}:")
                print(f" {'Job-Op':<12} {'Start':<12} {'End':<12} {'Duration':<12}")
                print(f" {'-'*50}")
           
            print(f" J{op['job']}-O{op['operation']:<10} "
                  f"{op['start']:<12.0f} "
                  f"{op['end']:<12.0f} "
                  f"{op['duration']:<12.0f}")
        
        print('\n' + "="*80)
        print(' SUMMARY')
        print("="*80)
        print(f"Status: {'OPTIMAL' if status == cp_model.OPTIMAL else 'FEASIBLE'}")
        print(f"Solve Time: {solver.WallTime():.2f}s")
        print(f"Total Operations: {len(schedule)}")
        print(f"Machines Used: {len(set(s['machine'] for s in schedule))}")
        print("="*80)
        
    else:
        print(f" No solution found. Status: {solver.StatusName(status)}")
